<a href="https://colab.research.google.com/github/walkerstipe/Surreal_GAN/blob/main/Multi_Task_NETWORK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#two 'heads' that each have different reward functions/targets
#similar to ensemble learning methinks but lets search around and see what there is (pretty sure ive done 
#a bunch of this shit before but can't recall)

#notes, actor critic, (ddpg particulalry), are really quite similar in some regards. 

#update: seems we dont want stopgradient (we want those gradients) and 'trainable' 
#is great but cant be adjusted after initialization, (immutable tensors)
#next trying to use copying of weights and inserting new layers (indexing ok prob?)
  #1. initialize layer with specific weights
  #2. insert layer into existing net, replacing previous layer

In [2]:
# #behold my sexy network diagram in asci. 
# # each input section is a task
# #1. subnetwork trained on a task or simple problem
# .  .  .  .      
# |   |  |  |    
# .  .  .  .       
# |   |  |  |    
# .  .  .  .     
# .  .  .  . 
# |   |  |  |
# .  .  .  . 
# |   |  |  |
# .  .  .  . 



# #"ensemble" of learners, (basically multiple personalities... literally though)
# .  .  .  .     .  .  .  .      .  .  .  .     .  .  .  .  
# |   |  |  |    |   |  |  |     |   |  |  |    |   |  |  |
# .  .  .  .     .  .  .  .      .  .  .  .     .  .  .  .  
# |   |  |  |    |   |  |  |     |   |  |  |    |   |  |  |
# .  .  .  .     .  .  .  .      .  .  .  .     .  .  .  .     
#                 .  .  .  .     .  .  .  .  
#                 |   |  |  |    |   |  |  |                 #feature extraction and mixing of the minds layer. grow this over time?
#                 .  .  .  .     .  .  .  .                  #synesthesia is a feature
#                 |   |  |  |    |   |  |  |                 #the curse of dimensionality has not been solved, (progress sure but not solved)
#                 .  .  .  .     .  .  .  . 
# .  .  .  .     .  .  .  .      .  .  .  .     .  .  .  .  
# |   |  |  |    |   |  |  |     |   |  |  |    |   |  |  |
# .  .  .  .     .  .  .  .      .  .  .  .     .  .  .  .  
# |   |  |  |    |   |  |  |     |   |  |  |    |   |  |  |
# .  .  .  .     .  .  .  .      .  .  .  .     .  .  .  .  

In [3]:
# #starting simple, can i interleave a MODEL into the hidden layers (middle bit) of another network? (and keep them
# #independent as models but still share weights)
# #"progressive growing gans" is similar example starting there

# #1. simple net
# .  .  .  .      
# |   |  |  |    
# .  .  .  .       
# |   |  |  |    
# .  .  .  .     
# .  .  .  . 
# |   |  |  |
# .  .  .  . 
# |   |  |  |
# .  .  .  . 

# #2. simple net with added model in the middle bit
# .  .  .  .      
# |   |  |  |    
# .  .  .  .       
# |   |  |  |    
# .  .  .  .     
# .  .  .  . 
# |   |  |  |
# .  .  .  . 
# |   |  |  |
# .  .  .  . 
# .  .  .  .      
# |   |  |  |    
# .  .  .  .       
# |   |  |  |    
# .  .  .  .     

In [2]:
# insert NETWORK (model) not just layer
# see if possible to keep as independent
from sklearn.datasets import make_blobs
import tensorflow
!pip install keras.utils
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from os import makedirs
from numpy.random import randn
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras import estimator
from keras import layers
from keras.models import Model

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=3, cluster_std=2, random_state=2)

# one hot encode output variable
y = to_categorical(y)

# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

# fit model
def make_mod(trainX, trainy):
  #model = Sequential()
  #my_input = keras.Input(shape=(28, 28, 1), name="img")
  my_input = keras.Input(shape=(3))#, name="img")
  #model.add(Dense(3, input_dim=3, activation='relu'))

  #x = layers.Conv2D(16, 3, activation="relu")(my_input)
  x_0 = layers.Dense(3, activation="relu")(my_input)
  x_1 = layers.Dense(3, activation="relu")(my_input)
  x_2 = layers.Dense(3, activation="relu")(my_input)
  #x = tf.stop_gradient(layers.Dense(3, activation="relu")(my_input))
  #model.add(Dense(3, input_dim=3, activation='relu'))
  
  #x = layers.Conv2D(32, 3, activation="relu")(x)
  my_output = layers.Dense(3, activation="softmax")(x_2)
  #model.add(Dense(3, activation='softmax'))
  my = keras.Model(my_input, my_output, name="my")
  my.summary()
  print("weights before training: ", my.layers[1].weights)

  yawg = []
  yawg.append(my.layers[1].weights)

  my.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  my.fit(trainX, trainy, epochs=50, batch_size=1, verbose=0)
#need to np.copy/deepcopy
#it's read only... wut?
  #is it just Sequential models that are immutable? swap to functional and all good?...

  #"All tensors are immutable like Python numbers and strings: you can never update 
  #the contents of a tensor, only create a new one."
  #possible to replace single layer?... (or is the whole thing considered a tensor together and thereby completely immutable...)
  my.layers[1].weights = yawg[0]
  print("weights after training: ", my.layers[1].weights)
  return my #model

model_0 = make_mod(trainX, trainy)
#tf.stop_gradient(model_0.layers[2].weights)
_, acc = model_0.evaluate(testX, testy, verbose=2)
print("weights after training: ", model_0.layers[1].weights)

print(acc)
model_1 = make_mod(trainX, trainy)
_, acc = model_1.evaluate(testX, testy, verbose=2)
print(acc)
print(model_0.layers[0].weights)


  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2656 sha256=be0d54032f58b49b8322360f0c1f6bb9c342268b8b7d47ed3bb696a55ef88965
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils
Model: "my"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 12        
Total params: 24
Trainable params: 24
Non-trainable params: 0
_________________________________________________________________
weights before training:  [<tf.Variable 'dense_2/kernel:0' shape=(3, 3) dtype=float32, numpy=
array(

AttributeError: ignored

In [7]:
layer1 = keras.layers.Dense(3, activation="relu")
layer2 = keras.layers.Dense(3, activation="sigmoid")
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])
print('before training: ', layer1.weights)

model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
print('after training: ', model.layers[1].weights)


#layer1_2 = keras.layers.Dense(3, activation="relu", weights = layer1.weights)
layer1_2 = keras.layers.Dense(3, activation="relu")
#layer2_2 = keras.layers.Dense(3, activation="sigmoid", weights = model.layers[1].weights)
layer2_2 = keras.layers.Dense(3, activation="sigmoid")

model_2 = keras.Sequential([keras.Input(shape=(3,)), layer1_2, layer2_2])

model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

before training:  [<tf.Variable 'dense_20/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.46883798, -0.29093933, -0.05204535],
       [-0.77176094, -0.02694535,  0.31962752],
       [-0.14452934,  0.2867167 , -0.19020152]], dtype=float32)>, <tf.Variable 'dense_20/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]
1/1 [==============================] - 1s 527ms/step - loss: 0.0481
after training:  [<tf.Variable 'dense_21/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.30250472, -0.3535108 , -0.9290188 ],
       [ 0.4720137 ,  0.6499977 , -0.17250347],
       [ 0.08599003, -0.37113804, -0.8853477 ]], dtype=float32)>, <tf.Variable 'dense_21/bias:0' shape=(3,) dtype=float32, numpy=array([ 0.00099987, -0.00099969, -0.00099993], dtype=float32)>]
1/1 [==============================] - 0s 481ms/step - loss: 0.0626


In [10]:
# Make a model with 2 layers
layer1 = keras.layers.Dense(3, activation="relu")
layer2 = keras.layers.Dense(3, activation="sigmoid")
tf.stop_gradient(layer2.weights)
#tf.stop_gradient(model_0.layers[0].weights)
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])
print('before training: ', layer1.weights)

model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
print('after training: ', model.layers[1].weights)

model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
print('no stop_grad: ', model.layers[1].weights)

model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
tf.stop_gradient(model.layers[1].weights)
print('with stop_grad: ', model.layers[1].weights)


before training:  [<tf.Variable 'dense_13/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.36368108,  0.66787887,  0.28092766],
       [-0.23105717, -0.83103824,  0.31745648],
       [ 0.56801176, -0.44976997,  0.9670644 ]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]
1/1 [==============================] - 1s 789ms/step - loss: 0.0806
after training:  [<tf.Variable 'dense_14/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.3515756 , -0.31720096, -0.88685733],
       [ 0.78182554,  0.9851663 ,  0.66817737],
       [-0.46885931, -0.68534786,  0.07711408]], dtype=float32)>, <tf.Variable 'dense_14/bias:0' shape=(3,) dtype=float32, numpy=array([-0.00099989,  0.00099987,  0.00099993], dtype=float32)>]
1/1 [==============================] - 0s 6ms/step - loss: 0.1144
no stop_grad:  [<tf.Variable 'dense_14/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.3509072 , -0.3165318 , -0.886188  ],
       [ 0

InvalidArgumentError: ignored

In [ ]:
tf.stop_gradient(model_0.layers[0].weights)

In [5]:
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights
layer.trainable = False  # Freeze the layer

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))


weights: 2
trainable_weights: 0
non_trainable_weights: 2


In [ ]:
# #original archived
# # Make a model with 2 layers
# layer1 = keras.layers.Dense(3, activation="relu")
# layer2 = keras.layers.Dense(3, activation="sigmoid")
# model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

# # Freeze the first layer
# layer1.trainable = False

# # Keep a copy of the weights of layer1 for later reference
# initial_layer1_weights_values = layer1.get_weights()

# # Train the model
# model.compile(optimizer="adam", loss="mse")
# model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# # Check that the weights of layer1 have not changed during training
# final_layer1_weights_values = layer1.get_weights()
# np.testing.assert_allclose(
#     initial_layer1_weights_values[0], final_layer1_weights_values[0]
# )
# np.testing.assert_allclose(
#     initial_layer1_weights_values[1], final_layer1_weights_values[1]
# )


In [6]:
# Make a model with 2 layers
layer1 = keras.layers.Dense(3, activation="relu")
layer2 = keras.layers.Dense(3, activation="sigmoid")
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

layer1.trainable = True

print(layer1.weights)
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

print(model.layers[1].weights)
model.layers[1].trainable = False 
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
print(model.layers[1].weights)

model.layers[1].trainable = False 
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
print(model.layers[1].weights)
#wut...


[<tf.Variable 'dense_5/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.05617666, -0.10382462, -0.5446832 ],
       [-0.1262505 ,  0.12848854,  0.88347125],
       [ 0.7048845 ,  0.33851194,  0.86311436]], dtype=float32)>, <tf.Variable 'dense_5/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]
1/1 [==============================] - 2s 2s/step - loss: 0.1439
[<tf.Variable 'dense_6/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.14261097,  0.8747211 ,  0.2584371 ],
       [-0.46983382,  0.0594026 , -0.5868757 ],
       [-0.8388236 , -0.5035944 ,  0.5075929 ]], dtype=float32)>, <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([ 0.00099994, -0.00099994, -0.00099985], dtype=float32)>]
1/1 [==============================] - 0s 6ms/step - loss: 0.0778
[<tf.Variable 'dense_6/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.14324905,  0.87386763,  0.25769845],
       [-0.46914893,  0.05856382, -0.58734673],
       [-0.83808696,

In [ ]:
# Make a model with 2 layers
layer1 = keras.layers.Dense(3, activation="relu")
layer2 = keras.layers.Dense(3, activation="sigmoid")
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

layer1.trainable = False

print(layer1.weights)
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

print(model.layers[0].weights)
layer1.trainable = False
model.layers[0].trainable = False 
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
print(model.layers[0].weights)

layer1.trainable = True
model.layers[0].trainable = True 
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
print(model.layers[0].weights)
#wut...


In [ ]:
print(layer1.weights)
print(model.layers[0].weights)

In [ ]:
print(model_0.layers[1].weights)

#x = np.copy(model_0.layers.weights)

In [ ]:
#why are the weight values changing... seems the variables might need to be 'copied'... shame
# array([[ 1.1752876 , -1.4872373 , -7.0670514 ],
#        [ 0.8589232 ,  0.78620785, -9.0647745 ],
#        [ 3.033589  , -2.9427311 , -2.1299865 ]]

In [ ]:
for layer in model_0.layers:
  print(layer)

In [ ]:
# #model_0.layers[1].output
# #model_0.add(Dense(3, activation='softmax'))
# model_0.add(model_1.layers[1])

# for layer in model_0.layers:
#   print(layer.input)
#   print(layer.output, '\n')

In [ ]:
#this cant be right... the output shape is supposed to be 3... it's 25 currently...
#_, acc = model_0.evaluate(testX, testy, verbose=2)

In [ ]:
#ha! i was right, shape mismatch. only shows up on 'fit'
# model_0.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model_0.fit(trainX, trainy, epochs=50, batch_size=1, verbose=0)
#behold imahgahd. if shapes match we can add layers from other model, (and keep them independent still)
#no probel ez pz. 

In [ ]:
#specify where to insert layer? create model entirely from other models layers? (doesn't seem to make sense. or necessary)
#bunch of useful tidbits here: https://stackoverflow.com/questions/49492255/how-to-replace-or-insert-intermediate-layer-in-keras-model
# Model = Sequential() #*** 2 types of models!! Sequential, (little changing possible), Functional, (way more dynamic)

# from keras.models import Model
# input=model_0.layers[0].input
# output=model_0.layers[1].output
# new_model = Model(input=model_0.layers[0].input, output=model_0.layers[1].output)

In [ ]:
model_0.layers[0]

In [ ]:
#new_model.add(model_0.layers[0])
my_input = keras.Input(shape=(3))
x = model_0.layers[1]
x = model_0.layers[0](x)
#my_output = model_0.layers[0](x)#basically this shouldn't be the output; should be a totally different output layer
my_output = layers.Dense(3, activation="softmax")(x)
my = keras.Model(my_input, my_output, name="my")
my.summary()
my.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
my.fit(trainX, trainy, epochs=50, batch_size=1, verbose=0)


# new_model.add(model_1.layers[0])
# new_model.add(model_0.layers[1])
# new_model.add(model_0.layers[0])
# new_model.add(model_1.layers[1])
# new_model.add(model_0.layers[1])
# new_model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
# new_model.fit(trainX, trainy, epochs=500, batch_size=1, verbose=0)#perfomance jumps dramatically with epoch size
_, acc = my.evaluate(testX, testy, verbose=2)
_, acc = model_1.evaluate(testX, testy, verbose=2)
print(my.layers)
# 32/32 - 0s - loss: nan - accuracy: 0.3380
# 32/32 - 0s - loss: 23.1159 - accuracy: 0.2580

In [ ]:
##Sequential version for posterities sake
# from keras.models import Model
# new_model = Sequential()
# # x = np.copy(model_0.layers[0])
# # new_model.add(x)
# #Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
# #but should be fine anyway
# #new_model.add(model_0.layers[0])
# new_model.add(model_1.layers[0])
# new_model.add(model_0.layers[1])
# new_model.add(model_0.layers[0])
# new_model.add(model_1.layers[1])
# new_model.add(model_0.layers[1])
# new_model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
# new_model.fit(trainX, trainy, epochs=500, batch_size=1, verbose=0)#perfomance jumps dramatically with epoch size
# _, acc = new_model.evaluate(testX, testy, verbose=2)
# _, acc = model_1.evaluate(testX, testy, verbose=2)
# print(new_model.layers)
# # 32/32 - 0s - loss: nan - accuracy: 0.3380
# # 32/32 - 0s - loss: 23.1159 - accuracy: 0.2580

In [ ]:
#new_model.add(model_0.layers[0])

for layer in new_model.layers:
  print(layer.input)
  
  print(layer.output, '\n')
#only adding layers if the'yre unique?? how strange

In [ ]:
# # print(model_0.layers[1].weights)
# import tensorflow as tf
# import keras
# from keras import estimator
# estimator_model = keras.estimator.model_to_estimator(keras_model=model_0)
# #model_copy_0 = tf.keras.models.clone_model(model_0)
# model_copy_0 = tf.keras.models.clone_model(model_0)

In [ ]:
print(model_0.layers[1].weights)
